# Lab 4 — Perceptron & Multi-Layer Perceptron (MLP)
**Name:** Jiya Raval &nbsp;|&nbsp; **USN:** 1AUA23BCS078 &nbsp;|&nbsp; **Class:** CSE-B2 &nbsp;|&nbsp; **Sem:** 6

---

## Task 1 — Single Layer Perceptron (from scratch)
**Objective:** Train a perceptron to classify points based on a linear boundary using Python from first principles.

**Steps:**
1. Setup Python Environment
2. Define Perceptron Function
3. Generate Synthetic Training Data
4. Train with Perceptron Learning Rule
5. Visualize Decision Boundary

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Training Data ---
X = np.array([[2,3],[1,1],[4,5],[6,7],[3,2],[7,8]])
y = np.array([0, 0, 0, 1, 1, 1])

# --- Initialize weights, bias, hyperparameters ---
w  = np.zeros(2)
b  = 0
lr = 0.1
epochs = 20

# --- Perceptron Learning Rule ---
for epoch in range(epochs):
    for i in range(len(X)):
        linear_output = np.dot(X[i], w) + b
        prediction    = 1 if linear_output >= 0 else 0
        error         = y[i] - prediction
        w += lr * error * X[i]
        b += lr * error

print("Learned Weights:", w)
print("Learned Bias   :", b)

In [ ]:
# --- Visualize Decision Boundary ---
colors = ['red' if label == 0 else 'blue' for label in y]
for i in range(len(X)):
    plt.scatter(X[i][0], X[i][1], color=colors[i], s=80, zorder=5)

x_vals = np.linspace(0, 8, 100)
y_vals = -(w[0] * x_vals + b) / w[1]

plt.plot(x_vals, y_vals, color='green', linewidth=2, label='Decision Boundary')
plt.scatter([], [], color='red',  label='Class 0')
plt.scatter([], [], color='blue', label='Class 1')
plt.title("Task 1: Single Layer Perceptron")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.legend()
plt.grid(True)
plt.show()

**Observations:**
- Implemented a single layer perceptron from scratch using NumPy.
- The model correctly separates linearly separable data using a straight decision boundary.
- A single layer perceptron **cannot** solve non-linear problems like XOR.

---
## Task 2 — Multi-Layer Perceptron (MLP) from scratch
**Objective:** Build an MLP classifier using NumPy to learn non-linear decision boundaries (XOR problem).

**Architecture:**
- Input layer  : 2 neurons
- Hidden layer : 4 neurons (ReLU activation)
- Output layer : 1 neuron (Sigmoid activation)

**Steps:** Define activations → Initialize weights → Forward pass → Backpropagation → Evaluate

In [ ]:
# --- Activation Functions ---
def relu(x):         return np.maximum(0, x)
def relu_grad(x):    return (x > 0).astype(float)
def sigmoid(x):      return 1 / (1 + np.exp(-x))
def sigmoid_grad(x): return x * (1 - x)

# --- XOR Dataset ---
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[0],[1],[1],[0]])

# --- Initialize Weights & Biases (hidden=4) ---
np.random.seed(42)
W1 = np.random.randn(2, 4) * 0.5
b1 = np.zeros((1, 4))
W2 = np.random.randn(4, 1) * 0.5
b2 = np.zeros((1, 1))

lr     = 0.1
epochs = 5000
losses = []

# --- Training Loop ---
for epoch in range(epochs):
    # Forward pass
    z1 = np.dot(X, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    output = sigmoid(z2)

    # Loss (MSE)
    loss = np.mean((y - output) ** 2)
    losses.append(loss)

    # Backpropagation
    d_out    = (output - y) * sigmoid_grad(output)
    dW2      = np.dot(a1.T, d_out)
    db2      = np.sum(d_out, axis=0, keepdims=True)
    d_hidden = np.dot(d_out, W2.T) * relu_grad(z1)
    dW1      = np.dot(X.T, d_hidden)
    db1      = np.sum(d_hidden, axis=0, keepdims=True)

    # Weight update
    W2 -= lr * dW2
    b2 -= lr * db2
    W1 -= lr * dW1
    b1 -= lr * db1

print("Final Predictions (Hidden=4):")
for inp, pred in zip(X, output):
    print(f"  Input: {inp}  →  Output: {pred[0]:.4f}  →  Predicted: {int(pred[0] > 0.5)}")

In [ ]:
# --- Plot Loss ---
plt.plot(losses, color='steelblue')
plt.title("Task 2: Loss vs Epochs (Hidden=4)")
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.grid(True)
plt.show()

**Observations:**
- MLP with 4 hidden neurons successfully learns the XOR problem.
- Loss decreases steadily over epochs, confirming backpropagation is working.
- Final predictions are very close to expected values `[0, 1, 1, 0]`.

---
## Task 3 — Comparison & Analysis
1. Compare Single Layer vs MLP on XOR (non-linear data)
2. Explore how hidden layer size impacts learning
3. Plot loss over training epochs for different hidden sizes

In [ ]:
# --- Helper: train MLP and return final loss + predictions ---
def train_mlp(hidden_size, epochs=5000, lr=0.1, seed=42):
    np.random.seed(seed)
    W1 = np.random.randn(2, hidden_size) * 0.5
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, 1) * 0.5
    b2 = np.zeros((1, 1))
    losses = []
    for _ in range(epochs):
        z1 = np.dot(X, W1) + b1;  a1 = relu(z1)
        z2 = np.dot(a1, W2) + b2;  o  = sigmoid(z2)
        losses.append(np.mean((y - o) ** 2))
        d_out    = (o - y) * sigmoid_grad(o)
        dW2      = np.dot(a1.T, d_out)
        db2      = np.sum(d_out, axis=0, keepdims=True)
        d_hidden = np.dot(d_out, W2.T) * relu_grad(z1)
        W2 -= lr * dW2;  b2 -= lr * db2
        W1 -= lr * np.dot(X.T, d_hidden)
        b1 -= lr * np.sum(d_hidden, axis=0, keepdims=True)
    return losses, o

# --- Compare hidden sizes: 2, 4, 8 ---
hidden_sizes = [2, 4, 8]
plt.figure(figsize=(12, 4))

for idx, hs in enumerate(hidden_sizes):
    loss_curve, preds = train_mlp(hs)
    plt.subplot(1, 3, idx + 1)
    plt.plot(loss_curve, color=['orange','steelblue','green'][idx])
    plt.title(f"Hidden Size = {hs}\nFinal Loss: {loss_curve[-1]:.4f}")
    plt.xlabel("Epochs"); plt.ylabel("MSE Loss")
    plt.grid(True)
    print(f"\n--- Hidden Size = {hs} ---")
    for inp, p in zip(X, preds):
        print(f"  {inp}  →  {p[0]:.4f}  (pred: {int(p[0]>0.5)})")

plt.suptitle("Task 3: Effect of Hidden Layer Size on XOR Learning", fontsize=12)
plt.tight_layout()
plt.show()

---
## Summary & Conclusions

| Model | XOR Problem | Linear Data |
|-------|------------|-------------|
| Single Layer Perceptron | ❌ Cannot solve | ✅ Works perfectly |
| MLP (hidden=2) | ⚠️ Struggles | ✅ Works |
| MLP (hidden=4) | ✅ Solves well | ✅ Works |
| MLP (hidden=8) | ✅ Solves best | ✅ Works |

- **Single Layer Perceptron** works only for linearly separable data.
- **MLP** with backpropagation can learn non-linear patterns like XOR.
- **Increasing hidden neurons** improves learning capacity but adds computation cost.
- **Hidden layers** allow the model to learn complex, non-linear decision boundaries.